# Calculating Expected Move for Multiple Options Contracts

* In the last tutorial we covered <b><a href = "https://github.com/yusifrefae/Jupyter-Projects/blob/main/stock-options-data-analysis.ipynb">How to Analyze Stock Options Data using Python</a></b>
* That was more of an educational/exploratory journey
* In this tutorial we'll build a more robust program that we can actually use day to day with minimal fiddling around

# The formula: 

# $$ \text{Expected Move} = P \times \text{IV} \times \sqrt{\frac{D}{365}} $$
#### P = Current Market Price
#### IV = Implied Volatility
#### D = Numer of Days until Expiration Date

### It's not as easy as it seems to write a program to do this. The formula is simple but the inputs are tricky
* In practice, options are arranged by expiration date, so we're going to pull the option chain for a single date
* P is very easy to get quickly, so we'll write a function to do this first, which I also covered in <b><a href="https://github.com/yusifrefae/Jupyter-Projects/blob/main/fetch-stock-data.ipynb">How To Fetch Stock Data</a></b>
* But IV must be the IV of an **At The Money** Put or Call
* So the next thing will be to write a function that takes the market price and finds the appropriate At-The-Money IV
* Finally, we'll need to convert the expiration to datetime and subtract the current date from it to find days to expiration
* The result will be a new 'Expected Move' column in the options chain, in addition to high and low stock price prediction columns

### We'll create a StockOptions class with various functions to accomplish this task

In [11]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
from yahooquery import Ticker
import datetime
from datetime import date, datetime

class StockOptions:
    def __init__(self, symbol):
        """Initialize the class with a stock symbol and load the option chain"""
        self.symbol = symbol
        self.ticker = Ticker(symbol)
        self.yf_ticker = yf.Ticker(symbol)
        
        # Load the option chain data once and store it in the object
        self.option_chain = self.ticker.option_chain.reset_index()
        
        # Convert expiration column to date to help with filtering
        self.option_chain['expiration'] = pd.to_datetime(self.option_chain['expiration']).dt.date
        
        # Add days to expiration column (expiration date - today's date)
        # Extracts the number of days from each timedelta object and floats the result
        today_date = datetime.now().date()
        self.option_chain['daysToExpiration'] = (self.option_chain['expiration'] - today_date).apply(lambda x: x.days).astype(float)

    def get_mkt_price(self):
        """Returns the current market price of a stock using yfinance library"""
        return self.yf_ticker.info.get('currentPrice')
    
    def get_atm_strike_price(self):
        """Returns the rounded current market price to approximate an at-the-money option"""
        return round(self.get_mkt_price())
    
    def get_exp_dates(self):
        """Returns a list of unique expiration dates of a stock from the loaded option chain"""
        # Find unique expiration dates from the option chain
        exp_dates = self.option_chain['expiration'].unique()        
        return exp_dates

    def print_exp_dates(self):
        """Prints an ordered list of unique expiration dates of a stock from the loaded option chain"""
        for i, date in enumerate(self.get_exp_dates(), start=1):
            print(f"{i}) {date}")

    def get_filtered_option_chain(self, exp_date, contract_type):
        """Returns a filtered dataframe of all options contracts with the given expiration date and contract type"""
        # Filter the option chain based on expiration date and contract type
        filtered_chain = self.option_chain[
            (self.option_chain['optionType'] == contract_type) &
            (self.option_chain['expiration'] == exp_date)] #pd.to_datetime(exp_date))]
        return filtered_chain
    
    def get_atm_iv(self, exp_date, contract_type):
        """Returns the implied volatility of the at-the-money option for a given expiration and contract type"""
        # Get the ATM strike price
        atm_strike = self.get_atm_strike_price()
        
        # Get the option chain filtered by expiration and contract type
        option_chain = self.get_filtered_option_chain(exp_date, contract_type)
        
        # Find the ATM implied volatility
        atm_iv = option_chain[option_chain['strike'] == atm_strike]['impliedVolatility'].iloc[0]
        return atm_iv

    def add_expected_move(self, exp_date, contract_type):
        """Adds columns for expected move, expected low, and expected high to the option chain"""
        # Get the filtered option chain
        df = self.get_filtered_option_chain(exp_date, contract_type).copy()
        
        # Get the current market price and add it as a column
        mkt_price = self.get_mkt_price()
        df['mktPrice'] = mkt_price
        
        # Get the ATM implied volatility and add it as a column
        atm_iv = self.get_atm_iv(exp_date, contract_type)
        df['atmIV'] = atm_iv
        
        # Calculate the expected move and add it as a column
        df['expectedMove'] = mkt_price * atm_iv * np.sqrt(df['daysToExpiration'] / 365)
        
        # Calculate the expected low and high prices and add them as columns
        df['expectedLo'] = mkt_price - df['expectedMove']
        df['expectedHi'] = mkt_price + df['expectedMove']
        return df

## That was a lot of code!! Let's break it down piece by piece and see how to use this class.

#### First we can creat a StockOptions object like this, and examine its various attributes

In [12]:
pfe = StockOptions("PFE")
print(f"This creates an object: {pfe}")
print(f"The class is: {type(pfe)}")
print(f"The symbol is: {pfe.symbol}")
print(f"The yahooquery Ticker object is: {pfe.ticker}")
print(f"The yfinance Ticker object is: {pfe.yf_ticker}")

This creates an object: <__main__.StockOptions object at 0x000002C9EDAC34D0>
The class is: <class '__main__.StockOptions'>
The symbol is: PFE
The yahooquery Ticker object is: <yahooquery.ticker.Ticker object at 0x000002C987AEB4D0>
The yfinance Ticker object is: yfinance.Ticker object <PFE>


#### We can also display the full options chain

In [13]:
pfe.option_chain

,symbol,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,lastTradeDate,impliedVolatility,inTheMoney,daysToExpiration
0,PFE,2025-10-03,calls,PFE251003C00015000,15.0,USD,9.85,0.00,0.000000,10.0,8,9.25,12.25,REGULAR,2025-09-30 19:14:39,6.003909,True,2.0
1,PFE,2025-10-03,calls,PFE251003C00018000,18.0,USD,5.68,0.00,0.000000,1.0,2,6.80,8.20,REGULAR,2025-09-26 15:07:13,0.000010,True,2.0
2,PFE,2025-10-03,calls,PFE251003C00019000,19.0,USD,6.07,0.00,0.000000,1.0,5,5.80,7.80,REGULAR,2025-09-30 17:32:19,3.089846,True,2.0
3,PFE,2025-10-03,calls,PFE251003C00019500,19.5,USD,5.50,0.00,0.000000,12.0,36,4.95,6.80,REGULAR,2025-09-30 17:06:01,0.000010,True,2.0
4,PFE,2025-10-03,calls,PFE251003C00020000,20.0,USD,5.50,0.00,0.000000,452.0,193,6.05,6.85,REGULAR,2025-09-30 19:18:55,1.406253,True,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,PFE,2028-01-21,puts,PFE280121P00025000,25.0,USD,4.20,-0.09,-2.097906,5.0,1440,3.55,4.05,REGULAR,2025-09-30 19:42:58,0.306525,False,842.0
591,PFE,2028-01-21,puts,PFE280121P00027000,27.0,USD,6.32,0.00,0.000000,75.0,518,4.80,5.90,REGULAR,2025-09-29 16:30:00,0.350837,True,842.0
592,PFE,2028-01-21,puts,PFE280121P00030000,30.0,USD,8.50,0.00,0.000000,2.0,41,6.00,7.40,REGULAR,2025-09-29 19:49:11,0.321418,True,842.0
593,PFE,2028-01-21,puts,PFE280121P00032000,32.0,USD,10.00,0.00,0.000000,30.0,74,7.80,11.00,REGULAR,2025-09-25 14:11:10,0.457159,True,842.0


#### Notice that the class converts the 'expiration' column from timestamp to date and adds a 'days to expiration' column
* The class then subtracts today's date from the expiration columns to create a 'days to expiration' column with float values
* If you scroll to the far right you'll notice the Oct 3, 2025 contracts have 2 days to expiration 
* The Jan 21, 2028 contracts have 842 days to expiration


#### We can get the current market price and the rounded market price (which is the approximate ATM strike price)

In [14]:
print(f"The current market price of {pfe.symbol} is ${pfe.get_mkt_price():.2f}")
print(f"The approximate ATM price of {pfe.symbol} is ${pfe.get_atm_strike_price():.2f}")

The current market price of PFE is $26.42
The approximate ATM price of PFE is $26.00


#### We can examine a list of every unique expiration date in the option chain

In [15]:
pfe.get_exp_dates()

array([datetime.date(2025, 10, 3), datetime.date(2025, 10, 10),
       datetime.date(2025, 10, 17), datetime.date(2025, 10, 24),
       datetime.date(2025, 10, 31), datetime.date(2025, 11, 7),
       datetime.date(2025, 11, 21), datetime.date(2025, 12, 19),
       datetime.date(2026, 1, 16), datetime.date(2026, 3, 20),
       datetime.date(2026, 6, 18), datetime.date(2026, 9, 18),
       datetime.date(2026, 12, 18), datetime.date(2027, 1, 15),
       datetime.date(2027, 12, 17), datetime.date(2028, 1, 21)],
      dtype=object)

#### We can iterate to print the dates in an ordered list, for better readability

In [16]:
pfe.print_exp_dates()

1) 2025-10-03
2) 2025-10-10
3) 2025-10-17
4) 2025-10-24
5) 2025-10-31
6) 2025-11-07
7) 2025-11-21
8) 2025-12-19
9) 2026-01-16
10) 2026-03-20
11) 2026-06-18
12) 2026-09-18
13) 2026-12-18
14) 2027-01-15
15) 2027-12-17
16) 2028-01-21


#### Ok now let's get fancy. We can filter the options data by type and date

In [17]:
pfe.get_filtered_option_chain(date(2025, 10, 31), 'puts')

,symbol,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,openInterest,bid,ask,contractSize,lastTradeDate,impliedVolatility,inTheMoney,daysToExpiration
215,PFE,2025-10-31,puts,PFE251031P00020000,20.0,USD,0.02,0.00,0.000000,20.0,113,0.00,0.07,REGULAR,2025-09-30 14:21:34,0.500005,False,30.0
216,PFE,2025-10-31,puts,PFE251031P00020500,20.5,USD,0.09,0.00,0.000000,0.0,1,0.00,0.08,REGULAR,2025-09-12 19:23:37,0.537114,False,30.0
217,PFE,2025-10-31,puts,PFE251031P00021000,21.0,USD,0.07,0.00,0.000000,5.0,79,0.00,0.06,REGULAR,2025-09-30 13:52:16,0.468755,False,30.0
218,PFE,2025-10-31,puts,PFE251031P00021500,21.5,USD,0.04,0.00,0.000000,50.0,92,0.00,0.10,REGULAR,2025-09-30 19:48:00,0.476568,False,30.0
219,PFE,2025-10-31,puts,PFE251031P00022000,22.0,USD,0.06,0.00,0.000000,30.0,189,0.01,0.07,REGULAR,2025-09-30 18:17:00,0.400397,False,30.0
220,PFE,2025-10-31,puts,PFE251031P00022500,22.5,USD,0.07,0.00,0.000000,41.0,214,0.00,0.07,REGULAR,2025-09-30 19:48:10,0.361335,False,30.0
221,PFE,2025-10-31,puts,PFE251031P00023000,23.0,USD,0.06,-0.03,-33.333336,22.0,720,0.05,0.07,REGULAR,2025-10-01 13:45:09,0.322272,False,30.0
222,PFE,2025-10-31,puts,PFE251031P00023500,23.5,USD,0.09,-0.05,-33.333330,7.0,1016,0.07,0.10,REGULAR,2025-10-01 13:46:43,0.308601,False,30.0
223,PFE,2025-10-31,puts,PFE251031P00024000,24.0,USD,0.14,-0.07,-33.333330,65.0,5489,0.12,0.14,REGULAR,2025-10-01 13:46:45,0.293952,False,30.0
224,PFE,2025-10-31,puts,PFE251031P00024500,24.5,USD,0.20,-0.14,-41.176470,17.0,351,0.14,0.21,REGULAR,2025-10-01 13:50:50,0.287117,False,30.0


#### We can find the implied volatility of an ATM option for a specific expiration date and option type

In [18]:
print(f"The ATM IV for {pfe.symbol} is {pfe.get_atm_iv(date(2025, 10, 31), 'puts')*100: .4f}%")

The ATM IV for PFE is  28.5652%


#### And finally, we can achieve our goal. We can add an expected move column to a filtered dataframe
* Note that this won't work on the entire dataframe, that would require more coding! And not be as useful as looking at specific dates
* Also, note that the class adds multiple helper columns (market price and atm IV) and output columns (expected high and low price) for easy reference

In [19]:
pfe.add_expected_move(date(2025, 10, 31), 'puts')

,symbol,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,percentChange,volume,...,contractSize,lastTradeDate,impliedVolatility,inTheMoney,daysToExpiration,mktPrice,atmIV,expectedMove,expectedLo,expectedHi
215,PFE,2025-10-31,puts,PFE251031P00020000,20.0,USD,0.02,0.00,0.000000,20.0,...,REGULAR,2025-09-30 14:21:34,0.500005,False,30.0,26.418,0.285652,2.16347,24.25453,28.58147
216,PFE,2025-10-31,puts,PFE251031P00020500,20.5,USD,0.09,0.00,0.000000,0.0,...,REGULAR,2025-09-12 19:23:37,0.537114,False,30.0,26.418,0.285652,2.16347,24.25453,28.58147
217,PFE,2025-10-31,puts,PFE251031P00021000,21.0,USD,0.07,0.00,0.000000,5.0,...,REGULAR,2025-09-30 13:52:16,0.468755,False,30.0,26.418,0.285652,2.16347,24.25453,28.58147
218,PFE,2025-10-31,puts,PFE251031P00021500,21.5,USD,0.04,0.00,0.000000,50.0,...,REGULAR,2025-09-30 19:48:00,0.476568,False,30.0,26.418,0.285652,2.16347,24.25453,28.58147
219,PFE,2025-10-31,puts,PFE251031P00022000,22.0,USD,0.06,0.00,0.000000,30.0,...,REGULAR,2025-09-30 18:17:00,0.400397,False,30.0,26.418,0.285652,2.16347,24.25453,28.58147
220,PFE,2025-10-31,puts,PFE251031P00022500,22.5,USD,0.07,0.00,0.000000,41.0,...,REGULAR,2025-09-30 19:48:10,0.361335,False,30.0,26.418,0.285652,2.16347,24.25453,28.58147
221,PFE,2025-10-31,puts,PFE251031P00023000,23.0,USD,0.06,-0.03,-33.333336,22.0,...,REGULAR,2025-10-01 13:45:09,0.322272,False,30.0,26.418,0.285652,2.16347,24.25453,28.58147
222,PFE,2025-10-31,puts,PFE251031P00023500,23.5,USD,0.09,-0.05,-33.333330,7.0,...,REGULAR,2025-10-01 13:46:43,0.308601,False,30.0,26.418,0.285652,2.16347,24.25453,28.58147
223,PFE,2025-10-31,puts,PFE251031P00024000,24.0,USD,0.14,-0.07,-33.333330,65.0,...,REGULAR,2025-10-01 13:46:45,0.293952,False,30.0,26.418,0.285652,2.16347,24.25453,28.58147
224,PFE,2025-10-31,puts,PFE251031P00024500,24.5,USD,0.20,-0.14,-41.176470,17.0,...,REGULAR,2025-10-01 13:50:50,0.287117,False,30.0,26.418,0.285652,2.16347,24.25453,28.58147


#### If you scroll to the far right, you'll notice that the expected move is 2.16
#### The predicted range for PFE stock on Oct 31 is 24.25 to 28.58
* These outputs will change dynamically as the stock price, volatility, and other inputs change in real-time
* Since these options all have the same expiration date, they all have the same expected move and expected price range

#### We can quickly do this for any other date, symbol or option type. Let's find the Nov 7 Expected Move for Verizon stock

In [20]:
df = StockOptions("VZ").add_expected_move(date(2025, 11, 7), 'puts')
df

,symbol,expiration,optionType,contractSymbol,strike,currency,lastPrice,change,percentChange,openInterest,...,lastTradeDate,impliedVolatility,inTheMoney,volume,daysToExpiration,mktPrice,atmIV,expectedMove,expectedLo,expectedHi
236,VZ,2025-11-07,puts,VZ251107P00037000,37.0,USD,0.06,0.00,0.000000,2,...,2025-09-26 19:25:32,0.585453,False,1.0,37.0,43.935,0.252449,3.531332,40.403668,47.466332
237,VZ,2025-11-07,puts,VZ251107P00038000,38.0,USD,0.15,0.00,0.000000,16,...,2025-09-29 16:21:54,0.318366,False,11.0,37.0,43.935,0.252449,3.531332,40.403668,47.466332
238,VZ,2025-11-07,puts,VZ251107P00039000,39.0,USD,0.18,0.00,0.000000,17,...,2025-09-30 17:54:51,0.353522,False,4.0,37.0,43.935,0.252449,3.531332,40.403668,47.466332
239,VZ,2025-11-07,puts,VZ251107P00041000,41.0,USD,0.40,0.00,0.000000,17,...,2025-09-30 19:35:00,0.299323,False,6.0,37.0,43.935,0.252449,3.531332,40.403668,47.466332
240,VZ,2025-11-07,puts,VZ251107P00042000,42.0,USD,0.81,0.00,0.000000,26,...,2025-09-29 19:55:50,0.255379,False,19.0,37.0,43.935,0.252449,3.531332,40.403668,47.466332
241,VZ,2025-11-07,puts,VZ251107P00043000,43.0,USD,1.00,0.00,0.000000,15,...,2025-09-30 19:49:33,0.250007,False,1.0,37.0,43.935,0.252449,3.531332,40.403668,47.466332
242,VZ,2025-11-07,puts,VZ251107P00044000,44.0,USD,1.42,-0.04,-2.739732,3,...,2025-09-30 19:49:45,0.252449,True,2.0,37.0,43.935,0.252449,3.531332,40.403668,47.466332
243,VZ,2025-11-07,puts,VZ251107P00045000,45.0,USD,2.30,0.00,0.000000,5,...,2025-09-30 14:11:33,0.263679,True,1.0,37.0,43.935,0.252449,3.531332,40.403668,47.466332


#### Expected move is 3.53 with a predicted range of 40.40 to 47.47... current market price is 43.94

### Now you can do real-time stock and options data analysis with minimal coding and keystrokes!!

## Author: Yusif Refae, MBA
#### Let's work together: <a href = "https://www.linkedin.com/in/yusifrefae/">Message me on LinkedIn!</a>